**Connect google drive**

In [4]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Thu Apr 22 20:00:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
'Colab Notebooks'  'My Drive'	 WcfCarage.rar
 Desktop.rar	   'Parallel '	 yolov3


**1) Clone the Darknet**



In [6]:
!git clone https://github.com/AlexeyAB/darknet

fatal: destination path 'darknet' already exists and is not an empty directory.


**2) Compile Darknet using Nvidia GPU**


In [7]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [8]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [9]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [10]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [11]:
!echo "Koala" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [12]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-04-22 20:01:28--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  45.8MB/s    in 3.7s    

2021-04-22 20:01:32 (41.6 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [13]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/0aug-image-_0_355555555555555555555555555555555.jpg  
  inflating: data/obj/0aug-image-_0_355555555555555555555555555555555.txt  
  inflating: data/obj/0aug-image-_0_355555555555555555555555555555555vertical.jpg  
  inflating: data/obj/0aug-image-_0_355555555555555555555555555555555vertical.txt  
  inflating: data/obj/1.jpg          
  inflating: data/obj/1.txt          
  inflating: data/obj/aug-image-_0_367.jpg  
  inflating: data/obj/aug-image-_0_367.txt  
  inflating: data/obj/aug-image-_0_367vertical.jpg  
  inflating: data/obj/aug-image-_0_367vertical.txt  
  inflating: data/obj/aug-image-_0_381.jpg  
  inflating: data/obj/aug-image-_0_381.txt  
  inflating: data/obj/aug-image-_0_381vertical.jpg  
  inflating: data/obj/aug-image-_0_381vertical.txt  
  inflating: data/obj/aug-image-_0_560.jpg  
  inflating: data/obj/aug-image-_0_560.txt  
  inflating: data/obj/aug-image-_0_560vertical.jpg  
  inflating: data/obj/aug-image-

In [14]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.8560112847222222 0.03153935185185185 0.08919270833333333 0.037037037037037035

['0', '0.8560112847222222', '0.03153935185185185', '0.08919270833333333', '0.037037037037037035']
0 data/obj/horz3.txt
0 0.8560112847222222 0.03153935185185185 0.08919270833333333 0.037037037037037035
0 0.854275173611111 0.9690393518518517 0.09049479166666666 0.043402777777777776

['0', '0.854275173611111', '0.9690393518518517', '0.09049479166666666', '0.043402777777777776']
1 data/obj/aug-image-_0_3991.txt
0 0.854275173611111 0.9690393518518517 0.09049479166666666 0.043402777777777776
0 0.9018012152777777 0.8059895833333333 0.08919270833333333 0.044849537037037035

['0', '0.9018012152777777', '0.8059895833333333', '0.08919270833333333', '0.044849537037037035']
2 data/obj/aug-image-_0_2880.txt
0 0.9018012152777777 0.8059895833333333 0.08919270833333333 0.044849537037037035
0 0.11577690972222221 0.09346064814814814 0.07964409722222222 0.041666666666666664

['0', '0.11577690972222221', '0.09346064814814814

In [15]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/aug-image-_0_2816.jpg', 'data/obj/vert3r90.jpg', 'data/obj/vert2r9018090.jpg', 'data/obj/aug-image-_0_5808vertical.jpg', 'data/obj/vert3r90180.jpg', 'data/obj/aug-image-_0_3991.jpg', 'data/obj/aug-image-_0_5552vertical.jpg', 'data/obj/aug-image-_0_9845.jpg', 'data/obj/aug-image-_0_3932.jpg', 'data/obj/aug-image-_0_3553.jpg', 'data/obj/aug-image-_0_2450vertical.jpg', 'data/obj/horz6r90180.jpg', 'data/obj/aug-image-_0_4457vertical.jpg', 'data/obj/horz2r90180.jpg', 'data/obj/aug-image-_0_8712vertical.jpg', 'data/obj/aug-image-_0_5405vertical.jpg', 'data/obj/aug-image-_0_8795.jpg', 'data/obj/aug-image-_0_9035.jpg', 'data/obj/aug-image-_0_4417.jpg', 'data/obj/aug-image-_0_6866.jpg', 'data/obj/aug-image-_0_6506.jpg', 'data/obj/aug-image-_0_4364.jpg', 'data/obj/vert1r9018090.jpg', 'data/obj/aug-image-_0_1222.jpg', 'data/obj/aug-image-_0_5910vertical.jpg', 'data/obj/horz4r9018090.jpg', 'data/obj/aug-image-_0_6492vertical.jpg', 'data/obj/aug-image-_0_1147.jpg', 'data/obj/aug-image-_0

In [16]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 89170, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.730822), count: 1, class_loss = 0.026391, iou_loss = 0.043621, total_loss = 0.070011 
 total_bbox = 89171, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000007, iou_loss = 0.000000, total_loss = 0.000007 
v3 (mse 